# Week 1: Mean/Covariance of a data set and effect of a linear transformation

In this week, we are going to investigate how the mean and (co)variance of a
dataset changes
when we apply affine transformation to the dataset.

## Learning objectives
1. Get Farmiliar with basic programming using Python and Numpy/Scipy.
2. Learn to appreciate implementing
   functions to compute statistics of dataset in vectorized way.
3. Understand the effects of affine transformations on a dataset.
4. Understand the importance of testing in programming for machine learning.

First, let's import the packages that we will use for the week

In [ ]:
# PACKAGE: DO NOT EDIT THIS CELL
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import time
import timeit
from numpy.testing import assert_allclose
from sklearn.datasets import fetch_olivetti_faces
matplotlib.style.use('fivethirtyeight')
%matplotlib inline

Next, we are going to retrieve Olivetti faces dataset.

When working with some datasets, before digging into further analysis, it is
almost always
useful to do a few things to understand your dataset. First of all, answer the
following
set of questions:

1. What is the size of your dataset?
2. What is the dimensionality of your data?

The dataset we have are usually stored as 2D matrices, then it would be really
important
to know which dimension represents the dimension of the dataset, and which
represents
the data points in the dataset.

__When you implement the functions for your assignment, make sure you read
the docstring for what each dimension of your inputs represents the data points,
and which
represents the dimensions of the dataset!__

In [ ]:
# Load faces data:
dataset = fetch_olivetti_faces(data_home='./')
faces = dataset.data

image_shape = (64, 64)

print(f'Shape of the faces dataset: {faces.shape}')
print(f'{faces.shape[0]} data points')

When your dataset are images, it's a really good idea to see what they look
like.

One very
convenient tool in Jupyter is the `interact` widget, which we use to visualize
the images (faces). For more information on how to use interact, have a look at
the documentation
[here](http://ipywidgets.readthedocs.io/en/stable/examples/Using%20Interact.html).

In [ ]:
from ipywidgets import interact

In [ ]:
def show_face(face):
    plt.figure()
    plt.imshow(face.reshape(image_shape), cmap='gray')
    plt.show()

In [ ]:
@interact(n=(0, len(faces) - 1))
def display_faces(n=0):
    plt.figure()
    plt.imshow(faces[n].reshape(image_shape), cmap='gray')
    plt.show()

## 1. Mean and Covariance of a Dataset

In [ ]:
# GRADED FUNCTION: DO NOT EDIT THIS LINE
def mean_naive(X):
    """Compute the sample mean for a dataset by iterating over the dataset.

    Arguments:
        X: Array of shape (N, D) representing the dataset.
           N is the size of the dataset, and D is the dimensionality of the dataset.

    Returns:
        mu: Array of shape (D,). Sample mean of the dataset X.
    """

    # Size of the dataset:
    N = X.shape[0]

    # Dimensionality of the dataset:
    D = X.shape[1]

    # Initialize the array corresponding to the mean:
    mu = np.zeros((D,))

    # Iterate over the dataset, and compute the mean:
    for i in range(N):
        mu += X[i, :]
    mu /= N

    return mu


def cov_naive(X):
    """Compute the sample covariance for a dataset by iterating over the dataset.

    Arguments:
        X: Array of shape (N, D) representing the dataset.
           N is the size of the dataset, and D is the dimensionality of the dataset.

    Returns:
        covariance: Array of shape (D, D). Sample covariance of the dataset X.
    """

    # Size of the dataset:
    N = X.shape[0]

    # Dimensionality of the dataset:
    D = X.shape[1]

    # Compute the mean:
    mu = mean_naive(X)

    # Initialize the covariance matrix:
    covariance = np.zeros((D, D))

    # Iterate over the dataset, and compute the covariance matrix:
    for j in range(D):
        for k in range(D):
            for i in range(N):
                covariance[j, k] += X[i, j] * X[i, k]
            covariance[j, k] -= N * mu[j] * mu[k]
    covariance /= N

    return covariance


def mean(X):
    """Compute the sample mean for a dataset.

    Arguments:
        X: Array of shape (N, D) representing the dataset.
           N is the size of the dataset, and D is the dimensionality of the dataset.

    Returns:
        mu: Array of shape (D,). Sample mean of the dataset X.
    """

    # Size of the dataset:
    N = X.shape[0]

    # Compute the mean:
    mu = np.sum(X, axis=0) / N

    return mu


def cov(X):
    """Compute the sample covariance for a dataset.

    Arguments:
        X: Array of shape (N, D) representing the dataset.
           N is the size of the dataset, and D is the dimensionality of the dataset.

    Returns:
        covariance: Array of shape (D, D). Sample covariance of the dataset X.
    """

    # Size of the dataset:
    N = X.shape[0]

    # Dimensionality of the dataset:
    D = X.shape[1]

    # Compute the mean:
    mu = mean(X).reshape((1, D))

    # Compute the covariance matrix:
    covariance = np.matmul(np.transpose(X), X)
    covariance -= N * np.matmul(np.transpose(mu), mu)
    covariance /= N

    return covariance

In [ ]:
# Test the implementation of 'mean':

# Test case 1:
X = np.array([[0., 1., 1.],
              [1., 2., 1.]])
expected_mean = np.array([0.5, 1.5, 1.])
assert_allclose(mean(X), expected_mean, rtol=1e-5)

# Test case 2:
X = np.array([[0., 1., 0.],
              [2., 3., 1.]])
expected_mean = np.array([1., 2., 0.5])
assert_allclose(mean(X), expected_mean, rtol=1e-5)

# Test case 3:
X = np.array([[0., 1.],
              [0., 1.]])
expected_mean = np.array([0., 1.])
assert_allclose(mean(X), expected_mean, rtol=1e-5)

In [ ]:
cov(np.array([[0., 1.],
              [1., 2.],
              [0., 1.],
              [1., 2.]]))

In [ ]:
# Test the implementation of 'cov':

# Test case 1:
X = np.array([[0., 1.],
              [1., 2.],
              [0., 1.],
              [1., 2.]])
expected_cov = np.array([[0.25, 0.25],
                         [0.25, 0.25]])
assert_allclose(cov(X), expected_cov, rtol=1e-5)

# Test case 2:
X = np.array([[0., 1.],
              [2., 3.]])
expected_cov = np.array([[1., 1.],
                         [1., 1.]])
assert_allclose(cov(X), expected_cov, rtol=1e-5)

# Test case 3:
X = np.array([[0., 1.],
              [0., 1.],
              [0., 1.]])
expected_cov = np.zeros((2, 2))
assert_allclose(cov(X), expected_cov, rtol=1e-5)

With the `mean` function implemented, let's take a look at the _mean_ face of
our dataset!

In [ ]:
def mean_face(faces):
    return faces.mean(axis=0).reshape(image_shape)


plt.imshow(mean_face(faces), cmap='gray');

One of the advantage of writing vectorized code is speedup gained when working
on larger dataset. Loops in Python
are slow, and most of the time you want to utilise the fast native code provided
by Numpy without explicitly using
for loops. To put things into perspective, we can benchmark the two different
implementation with the `%time` function
in the following way:

In [ ]:
X = np.random.randn(1000, 20)

# Benchmarking the time for computing the mean:
%time mean_naive(X)
%time mean(X)
pass

In [ ]:
# Benchmarking the time for computing the covariance:
%time cov_naive(X)
%time cov(X)
pass

## 2. Affine Transformation of Dataset
In this week we are also going to verify a few properties about the mean and
covariance of affine transformation of random variables.

Consider a data matrix $X$ of size (N, D). We would like to know
what is the covariance when we apply affine transformation $Ax_i + b$ for each
datapoint $x_i$ in $X$. i.e.
we would like to know what happens to the mean and covariance for the new
dataset if we apply affine transformation.

In [ ]:
# GRADED FUNCTION: DO NOT EDIT THIS LINE
def affine_mean(mu, A, b):
    """Compute the mean after an affine transformation.

    Arguments:
        mu: Array of shape (D,). Sample mean for some dataset.
        A: Array of shape (D, D).
        b: Array of shape (D,).

    Returns:
        affine_mu: Array of shape (D,). Sample mean after the affine transformation.
    """

    # Dimension of 'mu':
    D = len(mu)

    # Compute the mean after the affine transformation:
    affine_mu = np.matmul(A, mu.reshape((D, 1)))
    affine_mu = np.add(affine_mu, b.reshape((D, 1)))
    affine_mu = affine_mu.reshape((D,))

    return affine_mu

In [ ]:
# GRADED FUNCTION: DO NOT EDIT THIS LINE
def affine_covariance(S, A, b):
    """Compute the covariance matrix after an affine transformation.

    Arguments:
        S: Array of shape (D, D). Covariance matrix for some dataset.
        A: Array of shape (D, D).
        b: Array of shape (D,).
           This argument is useless here. However, I cannot remove it.

    Returns:
        affine_cov: Array of shape (D, D). Covariance matrix after the affine transformation.
    """

    # Dimensionality of the dataset:
    D = S.shape[0]

    # Compute the covariance matrix after the affine transformation:
    affine_cov = np.matmul(S, np.transpose(A))
    affine_cov = np.matmul(A, affine_cov)

    return affine_cov

In [ ]:
A = np.array([[0, 1],
              [2, 3]])
b = np.ones(2)
m = np.full((2,), 2)
S = 2 * np.eye(2)

# Test the implementation of 'affine_mean':
expected_affine_mean = np.array([3., 11.])
assert_allclose(affine_mean(m, A, b), expected_affine_mean, rtol=1e-4)

In [ ]:
# Test the implementation of 'affine_covariance':
expected_affine_cov = np.array([[2., 6.],
                                [6., 26.]])
assert_allclose(affine_covariance(S, A, b), expected_affine_cov, rtol=1e-4)

Once the two functions above are implemented, we can verify the correctness our
implementation. Assuming that we have some $A$ and $b$.

In [ ]:
random = np.random.RandomState(42)
A = random.randn(4, 4)
b = random.randn(4)

Next we can generate some random dataset $X$

In [ ]:
X = random.randn(100, 4)

Assuming that for some dataset $X$, the mean and covariance are $m$, $S$, and
for the new dataset after affine transformation $X'$, the mean and covariance
are $m'$ and $S'$, then we would have the following identity:

$$m' = \text{affine_mean}(m, A, b)$$

$$S' = \text{affine_covariance}(S, A, b)$$

In [ ]:
# Applying affine transformation once:
X1 = ((A @ (X.T)).T + b)
# Twice:
X2 = ((A @ (X1.T)).T + b)

One very useful way to compare whether arrays are equal/similar is use the
helper functions
in `numpy.testing`.

Check the Numpy
[documentation](https://docs.scipy.org/doc/numpy-1.13.0/reference/routines.testing.html)
for details.

If you are interested in learning more about floating point arithmetic, here is
a good [paper](http://citeseerx.ist.psu.edu/viewdoc/summary?doi=10.1.1.22.6768).

In [ ]:
assert_allclose(mean(X1), affine_mean(mean(X), A, b))
assert_allclose(cov(X1), affine_covariance(cov(X), A, b))

In [ ]:
assert_allclose(mean(X2), affine_mean(mean(X1), A, b))
assert_allclose(cov(X2), affine_covariance(cov(X1), A, b))